In [2]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [3]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [4]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [6]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [11]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [ ]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [ ]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [ ]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [ ]:
vid = ''
token = ''

In [ ]:
transaction = None
#Get 1 transaction
for message in txconsumer:
    transaction = message
    break
print(transaction)

In [ ]:
#Generate signature based on given data
signature = gen_signature(transaction.txid,transaction.payer, transaction.payee, transaction.amount, token)
print(signature)

In [ ]:
#Prepare data to be serialize
to_submit = {'vid': vid, 'txid': transaction.txid, 'signature': signature}
print(to_submit)

#Serialize data
data = serialize(submitschema, to_submit)
print(data)

#Submit data
producer.send('avro', data)

In [ ]:
result = None

#Get a result that matches with vid and txid
for message in txconsumer:
    if message.vid == vid and message.txid == transaction.txid:
        result = message
    break
print(result)